In [1]:
import torch
import numpy as np
import torchvision
from torch import nn
import sys
import os

import torchvision.transforms as transforms
import torchvision.datasets as datasets
import pandas as pd
import torch.optim as optim

from smithers.ml.vgg import VGG
from smithers.ml.utils import get_seq_model

import matplotlib.pyplot as plt

from smithers.ml.utils import randomized_range_finder
from smithers.ml.utils import randomized_svd
from smithers.ml.AHOSVD import AHOSVD

from time import time

import warnings
warnings.filterwarnings("ignore")


torch.cuda.empty_cache()
import datetime
import time



sys.path.insert(0, '../')
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')


start1 = time.time()



VGGnet = VGG(    cfg=None,
                 classifier='cifar',
                 batch_norm=False,
                 num_classes=10,
                 init_weights=False,
                 pretrain_weights=None)
VGGnet = VGGnet.to(device) #MODIF
VGGnet.make_layers()
VGGnet._initialize_weights()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(VGGnet.parameters(), lr=0.001, momentum=0.9)

VGGnet2 = VGG(    cfg=None,
                 classifier='cifar',
                 batch_norm=False,
                 num_classes=10,
                 init_weights=False,
                 pretrain_weights=None)
VGGnet = VGGnet.to(device) #MODIF
VGGnet.make_layers()
VGGnet._initialize_weights()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(VGGnet.parameters(), lr=0.001, momentum=0.9)





def save_checkpoint_torch(epoch, model, path, optimizer):
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()
            }, path)


def load_checkpoint(model, checkpoint_path):
    model.load_state_dict(torch.load(checkpoint_path)['model_state_dict'])










batch_size = 8 #this can be changed
data_path = '../datasets/' 
# transform functions: take in input a PIL image and apply this
# transformations
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
train_dataset = datasets.CIFAR10(root=data_path + 'CIFAR10/',
                                 train=True,
                                 download=True,
                                 transform=transform_train)
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_dataset = datasets.CIFAR10(root=data_path + 'CIFAR10/',
                                train=False,
                                download=True,
                                transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)
train_labels = torch.tensor(train_loader.dataset.targets).to(device)
targets = list(train_labels)









pretrained = '/u/s/szanin/Smithers/smithers/ml/tutorials/check_vgg_cifar10_60_v2.pth.tar' #Stefano's
model2 = VGGnet2
#load_checkpoint(model2, pretrained)
seq_model2 = get_seq_model(model2)
model2 = model2.to(device) #MODIF
seq_model2 = seq_model2.to(device) #MODIF


seq_model2.eval()

model = VGGnet
load_checkpoint(model, pretrained)
seq_model = get_seq_model(model)
model = model.to(device) #MODIF
seq_model = seq_model.to(device) #MODIF


seq_model.eval()


from smithers.ml.netadapter import NetAdapter

red_dim = 35 * 3 * 3
red_method = 'AHOSVD'
inout_method = 'FNN'
n_class = 10 

cutoff_idx = 7
netadapter = NetAdapter(cutoff_idx, red_dim, 'HOSVD', inout_method)
red_model = netadapter.reduce_net_AHOSVD(seq_model2, train_dataset, train_labels, train_loader, n_class, mode_list_batch = [25, 35, 3, 3], device = device)

print(f"l'inizzializzazione ha impiegato {(time.time()-start1)/60} minuti")

from smithers.ml.utils import compute_loss, train_kd
optimizer = torch.optim.Adam([{
            'params': red_model.premodel.parameters(),
            'lr': 1e-4
            }, {
            'params': red_model.proj_model.parameters(),
            'lr': 1e-5
            }, {
            'params': red_model.inout_map.parameters(),
            'lr': 1e-5
            }])

train_loss = []
test_loss = []
train_loss.append(compute_loss(red_model, device, train_loader))
test_loss.append(compute_loss(red_model, device, test_loader))

        
epochs = 60
filename = './cifar10_VGG16_RedNet'+'HOSVD'+\
            '_cutIDx_%d.pth'%(cutoff_idx)

for epoch in range(1, epochs + 1):                       #da qui alla fine era dentro l'else commentato
    print('EPOCH {}'.format(epoch), flush=True)
    train_loss.append(
            train_kd(red_model,
            model,
            device,
            train_loader,
            optimizer,
            train_max_batch=200,
            alpha=0.1,
            temperature=1.,
            epoch=epoch))
    test_loss.append(compute_loss(red_model, device, test_loader))

print(f"In tutto la procedura ha impiegato {(time.time()-start1)/60} minuti")
#torch.save([red_model.state_dict(), train_loss, test_loss], filename)
#torch.save([red_model.state_dict()], 'AHOSVD_temp.pth') #TEMPORANEO, SI PUÒ TOGLIERE


Loaded base model.


Loaded base model.

Files already downloaded and verified
Files already downloaded and verified
Initializing reduction. Chosen reduction method is: AHOSVD
Inizio forwarding dataset
La shape di out_model è torch.Size([50000, 256, 4, 4])
Fine forwarding dataset
La shape di tensor_red è torch.Size([50000, 35, 3, 3])
La shape di flattened_red_out_model è torch.Size([50000, 315])
Comincia ora il training della FNN
È terminato il training della rete neurale
l'inizzializzazione ha impiegato 10.719153467814127 minuti
Test Loss 5.526361733680964e-06
 Top 1:  Accuracy: 4962.0/50000 (9.92%)
Test Loss: 0.27631808668404817
Test Loss 4.710970412254333e-05
 Top 1:  Accuracy: 1057.0/10000 (10.57%)
Test Loss: 0.47109704122543333
EPOCH 1
Train Loss kd: 1.98233163356781e-05
Test Loss -0.0003493409291219711
 Top 1:  Accuracy: 7384.0/10000 (73.84%)
Test Loss: -3.493409291219711
EPOCH 2
Train Loss kd: 5.5418318510055545e-06
Test Loss -0.00044395178874969484
 Top 1:  Accuracy: 7943.0/10